In [1]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)




In [38]:
class RandomForest:
  def __init__(self,n_tree = 10,  min_samples_split=2, max_depth=10, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.n_tree = n_tree

  def fit(self,X,y):
    n_samples = X.shape[0]




    # itarete and create trees
    self.trees = []
    for i in range(self.n_tree):
      tree = DecisionTree(max_depth=self.max_depth,
                            min_samples_split=self.min_samples_split,
                            n_features=self.n_features)

      idxs = self._get_random_samp_with_replac(n_samples)

      X = X[idxs,:]
      y =y[idxs]
      tree.fit(X,y)

      self.trees.append(tree)

      print("----------------- ", i , " out of ", self.n_tree)


  def predict(self,X):
    preds = [tree.predict(X) for tree in self.trees]
    all_pred = np.swapaxes(preds,0,1)
    out = [self._most_common_label(t_pred) for t_pred in all_pred]
    return out

  def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value


  def _get_random_samp_with_replac(self,n_samples):
    idxs = np.random.choice(n_samples,n_samples,replace=True)
    return idxs

In [36]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
bc = datasets.load_breast_cancer()
X,y = bc.data,bc.target
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,shuffle=True)

In [44]:
clf = RandomForest(n_tree=100)
clf.fit(X_train,y_train)

-----------------  0  out of  100
-----------------  1  out of  100
-----------------  2  out of  100
-----------------  3  out of  100
-----------------  4  out of  100
-----------------  5  out of  100
-----------------  6  out of  100
-----------------  7  out of  100
-----------------  8  out of  100
-----------------  9  out of  100
-----------------  10  out of  100
-----------------  11  out of  100
-----------------  12  out of  100
-----------------  13  out of  100
-----------------  14  out of  100
-----------------  15  out of  100
-----------------  16  out of  100
-----------------  17  out of  100
-----------------  18  out of  100
-----------------  19  out of  100
-----------------  20  out of  100
-----------------  21  out of  100
-----------------  22  out of  100
-----------------  23  out of  100
-----------------  24  out of  100
-----------------  25  out of  100
-----------------  26  out of  100
-----------------  27  out of  100
-----------------  28  out of 

In [45]:
sum(clf.predict(X_test)==y_test)/len(y_test)

0.9440559440559441